In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
% matplotlib inline

plt.style.use('seaborn-whitegrid')

import warnings
warnings.filterwarnings('ignore')

In [ ]:
!wget https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv

--2021-04-18 01:24:50--  https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1922812 (1.8M) [text/plain]
Saving to: ‘data.csv’

data.csv            100%[===================>]   1.83M  --.-KB/s    in 0.04s   

2021-04-18 01:24:50 (46.9 MB/s) - ‘data.csv’ saved [1922812/1922812]



In [ ]:
df = pd.read_csv('/content/data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15002 entries, 0 to 15001
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   pickup_community_area   15000 non-null  float64
 1   fare                    15002 non-null  float64
 2   trip_start_month        15002 non-null  int64  
 3   trip_start_hour         15002 non-null  int64  
 4   trip_start_day          15002 non-null  int64  
 5   trip_start_timestamp    15002 non-null  int64  
 6   pickup_latitude         15000 non-null  float64
 7   pickup_longitude        15000 non-null  float64
 8   dropoff_latitude        14519 non-null  float64
 9   dropoff_longitude       14519 non-null  float64
 10  trip_miles              15002 non-null  float64
 11  pickup_census_tract     1 non-null      float64
 12  dropoff_census_tract    10761 non-null  float64
 13  payment_type            15002 non-null  object 
 14  company                 9862 non-null 

In [ ]:
df = df.drop(['trip_start_timestamp','trip_miles','pickup_census_tract',
              'dropoff_census_tract','trip_seconds','payment_type','tips', 
              'company','dropoff_community_area','pickup_community_area'], axis=1)
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14519 entries, 28 to 15000
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   fare               14519 non-null  float64
 1   trip_start_month   14519 non-null  int64  
 2   trip_start_hour    14519 non-null  int64  
 3   trip_start_day     14519 non-null  int64  
 4   pickup_latitude    14519 non-null  float64
 5   pickup_longitude   14519 non-null  float64
 6   dropoff_latitude   14519 non-null  float64
 7   dropoff_longitude  14519 non-null  float64
dtypes: float64(5), int64(3)
memory usage: 1020.9 KB


In [ ]:
df = df.drop(df[df['fare'] > 200].index)

df['trip_start_month'] = df['trip_start_month'].astype('str')
df['trip_start_hour'] = df['trip_start_hour'].astype('str')
df['trip_start_day'] = df['trip_start_day'].astype('str')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14518 entries, 28 to 15000
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   fare               14518 non-null  float64
 1   trip_start_month   14518 non-null  object 
 2   trip_start_hour    14518 non-null  object 
 3   trip_start_day     14518 non-null  object 
 4   pickup_latitude    14518 non-null  float64
 5   pickup_longitude   14518 non-null  float64
 6   dropoff_latitude   14518 non-null  float64
 7   dropoff_longitude  14518 non-null  float64
dtypes: float64(5), object(3)
memory usage: 1020.8+ KB


In [ ]:
np.random.seed(seed=2)
msk = np.random.rand(len(df)) < 0.85
traindf = df[msk]
evaldf = df[~msk]

print(len(traindf))
print(len(evaldf))

traindf.to_csv("train.csv", index=False, header=False)
evaldf.to_csv("eval.csv", index=False, header=False)

12347
2171


In [ ]:
X_train = traindf.drop(['fare','trip_start_month', 'trip_start_hour', 'trip_start_day'], axis=1)
X_test= evaldf.drop(['fare','trip_start_month', 'trip_start_hour', 'trip_start_day'], axis=1)
y_train = traindf['fare']
y_test = evaldf['fare']

In [ ]:
import xgboost as xgb

params = {
   
    'max_depth': 7,
    'gamma' :0,
    'eta':.03, 
    'subsample': 1,
    'colsample_bytree': 0.9, 
    'objective':'reg:linear',
    'eval_metric':'rmse',
    'silent': 0
}

In [ ]:
def XGBmodel(X_train,X_test,y_train,y_test,params):
    matrix_train = xgb.DMatrix(X_train,label=y_train)
    matrix_test = xgb.DMatrix(X_test,label=y_test)
    model=xgb.train(params=params,
                    dtrain=matrix_train,num_boost_round=5000, 
                    early_stopping_rounds=10,evals=[(matrix_test,'test')])
    return model

model = XGBmodel(X_train,X_test,y_train,y_test,params)

[01:30:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	test-rmse:14.3383
Will train until test-rmse hasn't improved in 10 rounds.
[1]	test-rmse:13.9656
[2]	test-rmse:13.6075
[3]	test-rmse:13.2647
[4]	test-rmse:12.9298
[5]	test-rmse:12.6048
[6]	test-rmse:12.2892
[7]	test-rmse:11.9906
[8]	test-rmse:11.7028
[9]	test-rmse:11.4191
[10]	test-rmse:11.1438
[11]	test-rmse:10.8827
[12]	test-rmse:10.6307
[13]	test-rmse:10.3801
[14]	test-rmse:10.1441
[15]	test-rmse:9.91638
[16]	test-rmse:9.69342
[17]	test-rmse:9.48145
[18]	test-rmse:9.27368
[19]	test-rmse:9.08069
[20]	test-rmse:8.89138
[21]	test-rmse:8.70842
[22]	test-rmse:8.52845
[23]	test-rmse:8.35854
[24]	test-rmse:8.19218
[25]	test-rmse:8.03533
[26]	test-rmse:7.88634
[27]	test-rmse:7.74017
[28]	test-rmse:7.6019
[29]	test-rmse:7.46952
[30]	test-rmse:7.33672
[31]	test-rmse:7.20876
[32]	test-rmse:7.09177
[33]	test-rmse:6.97854
[34]	test-rmse:6.86722
[35]	test-rmse:6.76291

In [ ]:
evaldf_dropped = evaldf.drop(['fare', 'trip_start_month', 'trip_start_hour', 'trip_start_day'], axis=1)
prediction = model.predict(xgb.DMatrix(evaldf_dropped), ntree_limit = model.best_ntree_limit).tolist()
prediction

[6.269515514373779,
 8.882007598876953,
 8.032068252563477,
 9.148283004760742,
 9.148283004760742,
 6.761546611785889,
 10.432635307312012,
 20.77995491027832,
 42.5172233581543,
 8.328774452209473,
 6.17222785949707,
 7.6026835441589355,
 10.967860221862793,
 7.56369161605835,
 7.56369161605835,
 7.196303367614746,
 12.095376014709473,
 5.789285182952881,
 8.205755233764648,
 7.511694431304932,
 9.777519226074219,
 7.955118179321289,
 8.055624961853027,
 7.448902130126953,
 10.07308578491211,
 6.554671764373779,
 9.756803512573242,
 22.50631332397461,
 7.342752456665039,
 7.16565465927124,
 6.409433841705322,
 14.281864166259766,
 12.057512283325195,
 10.519661903381348,
 9.21451187133789,
 20.608617782592773,
 21.390687942504883,
 6.493127346038818,
 8.485695838928223,
 13.073139190673828,
 8.150857925415039,
 14.947254180908203,
 7.641935348510742,
 18.43788719177246,
 9.643635749816895,
 6.653326034545898,
 13.775046348571777,
 8.09467887878418,
 8.450474739074707,
 9.696448326110